# AGLAE PIGE data treatment

In [ ]:
%pip install ipywidgets xlrd

In [ ]:
import bae
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import io
import pandas as pd
from pandas.api.typing import DataFrameGroupBy
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
uploader_report = widgets.FileUpload()
display(uploader_report)

In [ ]:
uploader_pige = widgets.FileUpload()
display(uploader_pige)

In [ ]:
uploader_pixe = widgets.FileUpload()
display(uploader_pixe)

In [ ]:
print(uploader_report.value)
print(uploader_pige.value)
print(uploader_pixe.value)

report = pd.read_excel(io.BytesIO(uploader_report.value[0].content))
pige_table = pd.read_csv(io.BytesIO(uploader_pige.value[0].content), sep="\t")
pixe_table = pd.read_excel(io.BytesIO(uploader_pixe.value[0].content),sheet_name="S_Conc. ppm", header=1, index_col=[0, 1])

# On garde les tableaux avec uniquement les standards

In [ ]:
stds = pd.DataFrame(
    {
         "Nom std": ["BrillA", "BrillB", "BrillC", "BrillD", "BG4", "BG3"],
         "Conc theorique": [143900, 172600, 12000, 10700, 50000, 75000],
    }
)

nb_stds = len(stds)

report_stds = report[report["Ref Objet"].isin(stds["Nom std"])]
pige_stds = pige_table[pige_table["Nom Fichier"].str.contains("|".join(stds["Nom std"]), regex=True)]

# conversion des colonnes de durées
#report_stds["Time"] = pd.to_timedelta(report_stds["Time"])
#report_stds["Analyse time"] = pd.to_timedelta(report_stds["Analyse time"])

# on vérifie que les tableaux report et pige_stds ont bien un nombre de lignes multiple du nombre de standards (6 en général)

if len(report_stds)%nb_stds == 0:
    print(f"youpi: la longueur du tableau report est bien un multiple de {nb_stds}")
else:
    print("fatal error: problème de longueur du tableau report")

if len(pige_stds)%nb_stds == 0:
    print(f"youpi: la longueur du tableau pige_stds est bien un multiple de {nb_stds}")
else:
    print("fatal error: problème de longueur du tableau pige_stds")
    

# Calcul Aire Gaussienne / Dose
On calcule l'aire sur dose pour tracer la courbe de calibration PIGE

In [ ]:
nb_analyses = int(len(report_stds)/nb_stds)

ad_stds = pd.DataFrame(
    {
        "Nom std": report_stds["Ref Objet"], 
        "A/D": pige_stds["Aire Gaussien"]/report_stds["Dose"], 
    }
)

# On crée une liste de dataframe pour chaque analyse de standards de la journée, avec et sans Pb
ad_stds_sansPb = []
ad_stds_avecPb = []

# On fait une boucle for pour parcourir le tableau ad par bloc du nombre de standards et extraire ceux avec du Pb et ceux sans
for i in range(0, len(ad_stds), nb_stds):
    ad_stds_i = ad_stds[i:i+nb_stds]
    ad_stds_sansPb.append(ad_stds_i[ad_stds_i["Nom std"].isin(["BrillA","BrillB","BrillD"])])
    ad_stds_avecPb.append(ad_stds_i[ad_stds_i["Nom std"].isin(["BrillC","BG4","BG3"])])

conc_stds_sansPb = stds[stds["Nom std"].isin(["BrillA","BrillB","BrillD"])]
conc_stds_avecPb = stds[stds["Nom std"].isin(["BrillC","BG4","BG3"])]


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,4))

# Fonction qui va afficher les courbes de calibration et son fit linéaire par méthode des moindres carrés ordinaires
def calib(titre, ad, conc, plot_idx):
    axes[plot_idx].set_title(titre)
    axes[plot_idx].set_xlabel("Aire / Dose (u.a.)")
    axes[plot_idx].set_ylabel("Concentration théorique (ppm)")
    axes[plot_idx].grid(True)
    color = ["blue", "red"]
    coefs = []
    
    # Boucle for pour afficher toutes les droites de calibration sans plomb
    for i, e in enumerate(ad):
        x = e["A/D"]
        y = conc["Conc theorique"]    
                  
        axes[plot_idx].plot(e["A/D"],conc["Conc theorique"], '.', label=f"Analyse {i}", markersize=12, color=color[i]) 
        
        # Ajustement d'une droite passant par l'origine (y = ax)
        coef, _, _, _ = np.linalg.lstsq(np.array(x).reshape(-1, 1), np.array(y), rcond=None)
        coefs.append(coef[0])
        
        # Prédictions de y pour cette droite ajustée
        y_pred = coef[0] * np.array(x)
    
        # Calcul du coefficient de détermination R²
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r_squared = 1 - (ss_res / ss_tot)
    
        # Affichage de la droite ajustée
        axes[plot_idx].plot(x, y_pred, label=f"Droite {i}: y={coef[0]:.2f}x\nR²={r_squared:.2f}", linestyle='--')
        
    axes[plot_idx].legend(loc='best', fontsize=10)
    return coefs

coefs_sansPb = calib("Courbe de calibration Na PIGE sans Plomb", ad_stds_sansPb, conc_stds_sansPb, 0)
coefs_avecPb = calib("Courbe de calibration Na PIGE avec Plomb", ad_stds_avecPb, conc_stds_avecPb, 1)

# Calcul de la correction de l'heure (régression linéaire) si besoin

time_sansPb = []
time_avecPb = []

# Moyenne des temps d'acquisition des standards avec ou sans Pb (en secondes)
for i in range(0, len(report_stds), nb_stds):
    report_stds_i =  pd.to_timedelta(report_stds["Time"][i:i+nb_stds])
    time_sansPb.append(report_stds_i[report_stds["Ref Objet"].isin(["BrillA","BrillB","BrillD"])].mean())
    time_avecPb.append(report_stds_i[report_stds["Ref Objet"].isin(["BrillC","BG4","BG3"])].mean())
   
# Calcul du coefficient k avec ou sans Pb

t_diff_sansPb = time_sansPb[-1] - time_sansPb[0]
t_diff_avecPb = time_avecPb[-1] - time_avecPb[0]

coef_diff_sansPb = coefs_sansPb[-1] - coefs_sansPb[0]
coef_diff_avecPb = coefs_avecPb[-1] - coefs_avecPb[0]


k_sansPb = coef_diff_sansPb / t_diff_sansPb.seconds
k_avecPb = coef_diff_avecPb / t_diff_avecPb.seconds
# print(k_sansPb, k_avecPb)


In [ ]:
# import de la colonne des teneurs en PbO du fichier PIXE

#pixe_table_PbO = pixe_table.filter(like="PbO")
#pixe_table_PbO = pixe_table["PbO"]

pixe_table_PbO = pixe_table["PbO"].reset_index(drop=True)

report_time = pd.to_timedelta(report["Time"])
k_t_sansPb = k_sansPb * report_time.dt.total_seconds() + coefs_sansPb[-1] - k_sansPb * time_sansPb[-1].seconds
k_t_avecPb = k_avecPb * report_time.dt.total_seconds() + coefs_avecPb[-1] - k_avecPb * time_avecPb[-1].seconds

ad = pige_table["Aire Gaussien"] / report["Dose"]

Na_conc_df = pd.DataFrame(
    {
        "Ref Objet": report["Ref Objet"],
        "PbO": pixe_table_PbO, 
        "Time": report["Time"],
        "k(t) sans Pb": k_t_sansPb,
        "k(t) avec Pb": k_t_avecPb,
        "A/D": ad,
        "Na2O PIGE": np.where(
                pixe_table_PbO < 5000, 
                k_t_sansPb * ad,  
                k_t_sansPb * ad
            ),
    }
)

# l'index est un tuple dans pixe_table donc on extrait que les valeurs de Na_conc_df
pixe_table["Na2O PIGE"] = Na_conc_df["Na2O PIGE"].values
pixe_Na2O = pixe_table["Na2O"]
pixe_table = pixe_table.drop(columns=["Na2O"])
pige_corrected_compositions_table = pixe_table
pige_corrected_compositions_table

In [ ]:
pd.set_option("display.float_format", "{:.2f}".format)

Normalisation du fichier PIXE PIGE

In [ ]:
# Fonction qui permet de normaliser par rapport au Na2O PIGE

def normalize(row: pd.Series):
    cleaned_row = row.map(lambda s: pd.to_numeric(s, downcast="integer", errors="coerce"))
    sodium_pige = cleaned_row.loc["Na2O PIGE"]
    total_comp_without_Na = 1000000 - sodium_pige
    row_without_Na = cleaned_row.drop("Na2O PIGE")
    sum_comp_without_Na = row_without_Na.sum()
    return pd.concat([pd.Series(sodium_pige, index=["Na2O PIGE"]), row_without_Na.map(lambda x: x * total_comp_without_Na / sum_comp_without_Na)])

normalized_compositions_table = pige_corrected_compositions_table.apply(normalize, axis=1)
normalized_compositions_table

In [ ]:
def extract_simplified_name(row):
    name = row[1].strip().replace(" ", "_")

    if "_" in name:
        split = name.rsplit('_', 1)
        return split[0] if "pt" in split[1] or "map" in split[1] else name
    else:
        return name

x_widget = widgets.FloatSlider(min=0.2, max=3.0, step=0.1)
@interact(df=fixed(normalized_compositions_table), tolerance=x_widget, threshold=fixed(10000))
def remove_outliners(df: pd.DataFrame, tolerance, threshold=10000):
    to_delete=[]
    major_group = df[normalized_compositions_table > threshold].groupby(extract_simplified_name, sort=False)
    for name, grp in major_group:
        if grp.count().max() <= 1:
            continue

        median = grp.median()
        d = (grp - median) / median
        for i, row in d.iterrows():
            for j, elt in row[row > tolerance].items():
                print(i, j)
                to_delete.append(i)
                
    return to_delete

#normalized_compositions_table.loc[to_delete[0]]
#df_group.filter(detect_outliers)
#pager(res,5,10)
#pager(comp_table_without_outliners,90,10)
#for idx in to_delete:
#    print(idx[2])


In [ ]:
table_without_outliners = normalized_compositions_table.drop(remove_outliners(normalized_compositions_table, tolerance=0.5))

In [ ]:
final_table_ppm = table_without_outliners.groupby(extract_simplified_name, sort=False).agg(["mean", "std"])
final_table_ppm

In [ ]:
major_elements_list = ["Na2O PIGE", "MgO", "Al2O3", "SiO2", "P2O5", "SO3", "Cl", "K2O", "CaO", "TiO2","V2O3", "Cr2O3", "MnO", "Fe2O3", "CoO", "NiO", "CuO", "ZnO", "As2O5", "SrO", "Ag2O", "SnO2", "Sb2O5", "BaO", "Au2O3", "PbO"]

In [ ]:
final_table_ppm_major_elements = final_table_ppm[major_elements_list]
final_table_ppm_major_elements.head(5)

In [ ]:
final_table_wt = (final_table_ppm / 10000).round(3)
final_table_wt.head(5)

In [ ]:
final_table_wt_major_elts = final_table_wt[major_elements_list]

In [ ]:
with pd.ExcelWriter("final.xlsx") as writer:
    final_table_ppm.map(lambda x: f"{x:.0f}").to_excel(writer, sheet_name="ppm all elements")
    final_table_ppm_major_elements.map(lambda x: f"{x:.0f}").to_excel(writer, sheet_name="ppm major elements")
    final_table_wt.to_excel(writer, sheet_name="wt all elements")
    final_table_wt_major_elts.to_excel(writer, sheet_name="wt major elements")